In [4]:
import os
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
# jupyter 환경에서 asyncio를 사용할 수 있도록 설정
# 이는 Jupyter Notebook에서 비동기 작업을 지원하기 위해 필요합니다.
nest_asyncio.apply()

In [5]:
import os
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

documents = LlamaParse(result_type="markdown")


def pdf_parser(pdf_file_path: str):
    """
    PDF 파일을 파싱하여 그 내용을 Markdown 파일로 저장합니다.

    Args:
        pdf_file_path (str): 처리할 PDF 파일의 경로.
    """
    print(f"🔄 '{pdf_file_path}' 파일 파싱을 시작합니다...")

    # parsing instruction 을 지정합니다.
    parsing_instruction = (
        "You are parsing a AI Report. Please extract tables in markdown format."
    )

    # LlamaParse 설정
    parser = LlamaParse(
        use_vendor_multimodal_model=True,
        vendor_multimodal_model_name="openai-gpt4o",
        vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
        result_type="markdown",
        language="ko",
        parsing_instruction=parsing_instruction,
    )

    # 1. LlamaParse를 사용하여 PDF 파일을 로드합니다.
    # 'documents' 객체는 이 함수 외부에서 미리 정의되어 있어야 합니다.
    parsed_docs = documents.load_data(file_path=pdf_file_path)

    # 2. LangChain 형식의 도큐먼트로 변환합니다.
    docs = [doc.to_langchain_format() for doc in parsed_docs]

    return docs

In [6]:
# --- 함수 사용 예시 ---
# 이 코드를 실행하기 전에 'documents' 파서 객체를 초기화해야 합니다.
# file_to_parse = "data/디지털정부혁신추진계획.pdf"
file_to_parse = "./data/appendix-keywords-edit.pdf"
docs = pdf_parser(file_to_parse)

🔄 './data/appendix-keywords-edit.pdf' 파일 파싱을 시작합니다...
Started parsing the file under job_id e8c07a61-6e1c-4c5b-b655-8fca0d16bd08


<string>:8: RuntimeWarning: coroutine 'LlamaParse.aload_data' was never awaited


In [4]:
docs[0]  # .page_content

Document(id='7b6feac9-6a27-4856-8aa9-eb9d36bd7b07', metadata={}, page_content='써니행정법 공단기\n\n1. 구 식품위생법은 보건사회부장관(현 보건복지부장관)이 지정하여 고시(告示)하는 영업 또는 품목의 경우는 영업허가를 제한할 수 있다고 규정하였고, 이에 따라 보건사회부장관은 “그 전량을 수출하거나 주한 외국인에게만 판매한다는 요건을 갖춘 경우에만 보존음료수제조업의 허가를 할 수 있다.”라는 고시를 발한 바 있었다. 이 고시에 대한 설명으로 옳은 것은? 2010 국가직 9급\n\n1. ① 위 고시의 법적 성질을 행정규칙이라고 보는 것이 대법원의 입장이다.\n2. ② 위 고시에 정한 허가기준에 따라 보존음료수제조업허가에 붙여진 전량수출 또는 주한 외국인에 대한 판매에 한한다는 내용의 조건에 대해서는 행정행위에 부관을 붙일 수 있는 한계에 관한 일반원칙이 적용되지 않는다.\n3. ③ 위 고시상의 조건을 위반한 행위에 대하여 행정청이 과징금을 부과한 제재적 행정처분은 위법하지 아니하다.\n4. ④ 대법원은 행정청이 甲에 대하여 보존음료수제조업허가를 하면서 붙인 위 허가조건이 甲의 영업의 자유의 본질적 내용을 침해한다고 볼 수 없다고 하였다.\n\n# 해설\n\n① × 이른바 법령보충규칙으로서 법규명령이라는 것이 판례의 입장이다.\n\n식품제조영업허가기준이라는 고시는 공익상의 이유로 허가를 할 수 없는 영업의 종류를 지정할 권한을 부여한 구 식품위생법 제23조의3 제4호에 따라 보건사회부장관(현 보건복지부장관)이 발한 것으로서, 실질적으로 법의 규정내용을 보충하는 기능을 지니면서 그것과 결합하여 대외적으로 구속력 이 있는 법규명령의 성질을 가진 것이다(대판 1994. 3. 8, 92누1728).\n\n② ○ 법정부관의 경우 부관과는 구별되는 것으로 부관의 한계에 관한 일반적인 원칙이 적용되지 않는다는 것이 판례의 입장이다.\n\n고시에 정한 허가기준에 따라 보존음료수제조업의 허가에 붙여진 전량 수출 또는 

In [6]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",  # 문장을 나누는 구분자 설정
    chunk_size=250,  # 각 청크의 최대 길이 설정
    chunk_overlap=0,  # 청크 간의 겹치는 부분 설정
    length_function=len,  # 길이를 계산하는 함수 설정
)

In [7]:
myDocs = text_splitter.split_documents(docs)

Created a chunk of size 313, which is longer than the specified 250
Created a chunk of size 331, which is longer than the specified 250
Created a chunk of size 469, which is longer than the specified 250
Created a chunk of size 289, which is longer than the specified 250
Created a chunk of size 816, which is longer than the specified 250


In [8]:
myDocs[0]

Document(metadata={}, page_content='써니행정법 공단기\n\n1. 구 식품위생법은 보건사회부장관(현 보건복지부장관)이 지정하여 고시(告示)하는 영업 또는 품목의 경우는 영업허가를 제한할 수 있다고 규정하였고, 이에 따라 보건사회부장관은 “그 전량을 수출하거나 주한 외국인에게만 판매한다는 요건을 갖춘 경우에만 보존음료수제조업의 허가를 할 수 있다.”라는 고시를 발한 바 있었다. 이 고시에 대한 설명으로 옳은 것은? 2010 국가직 9급')